# 从数据库获取数据

In [1]:
# 导入数据
import pandas as pd
import pymysql
db = pymysql.connect(host="localhost", user="root", passwd='lvoe07', db="lagou_crawl", charset='utf8')
sql = '''
    select positionName,companyShortName,
    min_salary,max_salary,avg_salary,
    companyLabelList,job_description,industryField,companySize,
    city,education,workyear,firstType
    from lagou_crawl.position_info;
    '''
data = pd.read_sql(sql, db)
data['count'] = 1
db.close()

In [2]:
data

,positionName,companyShortName,min_salary,max_salary,avg_salary,companyLabelList,job_description,industryField,companySize,city,education,workyear,firstType,count
0,数据挖掘工程师,我来贷WeLab,25.0,35.0,30.0,"年底双薪,五险一金,带薪年假,扁平管理",一、数据挖掘工程师岗位职责:1、负责数据仓库的创建和管理，创建ETL平台，负责将原始数据转换...,金融,150-500人,深圳,本科,3-5年,技术,1
1,数据挖掘工程师,陌陌,20.0,40.0,30.0,"岗位晋升,帅哥多,管理规范,技能培训",工作职责：1、基于大规模用户数据，运用机器学习/数据挖掘的理论和方法，建立和优化通用或特定领...,社交网络,500-2000人,北京,本科,1-3年,技术,1
2,数据挖掘,悠易,20.0,35.0,27.5,"节日礼物,技能培训,绩效奖金,年度旅游",职责：1．基于海量数据的广告业务数据分析；2．根据广告反馈数据，对广告投放进行数据建模，包括...,广告营销,150-500人,北京,本科,不限,技术,1
3,数据挖掘,京东金融,20.0,35.0,27.5,"供应链金融,消费金融,众筹业务,支付业务",职位描述:1、构建风险计量模型对业务风险进行定性与定量评估；2、建立针对用户/商户/交易欺诈...,金融,2000人以上,北京,本科,5-10年,技术,1
4,数据挖掘,通联数据,15.0,30.0,22.5,"五险一金,通讯津贴,带薪年假,节日礼物",岗位职责1. 利用机器学习和NLP技术从海量异构数据中进行有价值金融信息的挖掘和整合；...,金融,150-500人,北京,本科,1-3年,技术,1
5,数据挖掘研发工程师,酷云互动,20.0,40.0,30.0,"技能培训,节日礼物,年度旅游,丰盛的早午餐",岗位职责： 运用机器学习、数据挖掘技术判别识别 主要工作内容包括： 1.通过对数据的敏...,数据服务,150-500人,北京,本科,3-5年,技术,1
6,数据挖掘,爱云校,15.0,30.0,22.5,"技能培训,节日礼物,股票期权,带薪年假",工作职责：1、根据需求文档完成功能设计、开发与测试 2、分析与解决线上问题，定位线上服务瓶颈...,教育,150-500人,北京,本科,1-3年,技术,1
7,数据挖掘工程师,齐家网,15.0,30.0,22.5,"午餐补助,带薪年假,绩效奖金,交通补助",岗位职责：1．搭建大规模数据挖掘平台，开发创新性算法和技术；2．根据需求，制定数据挖掘项目计...,O2O,2000人以上,上海,硕士,3-5年,技术,1
8,数据挖掘/机器学习工程师,360,25.0,35.0,30.0,null,【奇虎360搜索团队】【机器学习/数据挖掘工程师】-北京工作职责：负责query意图分析、排...,游戏,2000人以上,北京,本科,3-5年,技术,1
9,数据挖掘工程师,美团点评,20.0,40.0,30.0,"技能培训,绩效奖金,岗位晋升,领导好",数据挖掘工程师工作职责：1、结合具体业务场景，发现潜在的特征模型，挖掘现有数据的价值；2、对...,O2O,2000人以上,北京,本科,3-5年,技术,1


## 根据行业进行分组,统计公司数量与岗位数量
<p>industry_top10 = ['金融', '移动互联网', '数据服务', '电子商务', 'O2O', '企业服务', '教育', '游戏', '文化娱乐', '社交网络']</p>
<p>job_count = [98, 66, 53, 47, 45, 27, 19, 18, 16, 15]</p>
<p>company_count = [57, 42, 23, 31, 18, 18, 10, 12, 11, 11]</p>
<p>job_per_com = [1.72, 1.57, 2.3, 1.52, 2.5, 1.5, 1.9, 1.5, 1.45, 1.36]</p>

### 出现最频繁的行业是金融

In [3]:
from pyecharts import Bar,Line,Overlap
# 按行业分类,按行业的出现频率降序排序
industry = data.groupby('industryField').agg({'positionName': pd.Series.count,'companyShortName': pd.Series.nunique})
industry = industry.sort_values(['positionName', 'companyShortName'], ascending=[0, 0])
# 取出出现最频繁的十个行业
industry_top10 = list(industry.index)[:10]
job_count = list(map(int, industry['positionName']))[:10]   # 职位计数
company_count = list(map(int, industry['companyShortName']))[:10]   # 公司计数
job_per_com = []    # 每个行业对该岗位的需求程度
for i in range(0, len(industry_top10)):
    job_per_com.append(round(job_count[i]/company_count[i], 2))

In [4]:
bar = Bar('行业分布TOP10', subtitle='行业-职位数/公司数')
bar.add("职位数量", x_axis=industry_top10, bar_category_gap=0,xaxis_interval = 0,
        y_axis=job_count, is_label_show=1)
bar.add("公司数量", is_label_show=1, datazoom_orient='vertical',
        x_axis=industry_top10, xaxis_rotate=-45, xaxis_interval = 0,
        y_axis=company_count, yaxis_name='职位数量',
        yaxis_force_interval=20, yaxis_name_gap=40,
        datazoom_type='both', bar_category_gap=20)

# 行业-job/com-Line,表明公司对数据分析的需求程度
line = Line()
line.add('', x_axis=industry_top10, y_axis=job_per_com,
        line_width=2, is_label_show=1,
        yaxis_max=5, yaxis_force_interval=1)


overlap = Overlap()
overlap.add(bar)
overlap.add(line, yaxis_index=1, is_add_yaxis=True)
overlap

In [5]:
# 行业分布 - 饼图
from pyecharts import Pie
ind = []
jcount = []
ccount = []
elsesum_j = 0
elsesum_c = 0
for i in range(0,len(industry_top10)):
    if job_count[i] >= 40:
        ind.append(industry_top10[i])
        jcount.append(job_count[i])
        ccount.append(company_count[i])
    else:
        elsesum_j += job_count[i]
        elsesum_c += company_count[i]
ind.append('其他')
jcount.append(elsesum_j)
ccount.append(elsesum_c)


pie = Pie('行业占比',subtitle = '行业-职位数',title_pos = 'center',title_top = '44%')
pie.add('',ind,jcount,radius=[40,75],is_label_show = 1)
pie

# 各行业排名前十的公司

In [6]:
inds = ['金融','移动互联网','数据服务','电子商务','O2O']
for ind in inds:
    df = data[data['industryField'] == ind].groupby('companyShortName').agg({'count':pd.Series.count})\
    .sort_values(['count'], ascending=[0])
    print([ind,list(df.index)[:5]])

['金融', ['蚂蚁金服集团', '无秘', '奇虎360金融', '网贷之家', '广州银行信用卡中心']]
['移动互联网', ['金山办公软件', '小米', '上海宏鹿信息技术服务有限公司', '最右', '滴滴出行']]
['数据服务', ['搜狗', '集奥聚合（GEO）', '今日头条', '极光', '百度']]
['电子商务', ['京东商城', '欢乐逛', '蜜芽', '同程旅游', '网易']]
['O2O', ['美团点评', '人人车', '链家', '首汽约车', '二维火']]


# 各城市频率统计

In [7]:
from pyecharts import Bar,Line,Overlap
# 按城市分类,对城市的出现次数进行降序排序
city = data.groupby('city').agg({'positionName': pd.Series.count,'companyShortName': pd.Series.nunique})
city = city.sort_values(['positionName', 'companyShortName'], ascending=[0, 0])
# 取出出现最频繁的十个城市
city_top10 = list(city.index)[:10]
job_count = list(map(int, city['positionName']))[:10]   # 职位计数
company_count = list(map(int, city['companyShortName']))[:10]   # 公司计数
job_per_com = []    # 每个行业对该岗位的需求程度
for i in range(0, len(city_top10)):
    job_per_com.append(round(job_count[i]/company_count[i], 2))
city

,positionName,companyShortName
city,,
北京,242,134
上海,69,55
杭州,35,22
深圳,34,23
广州,16,13
武汉,12,11
成都,11,8
厦门,8,6
南京,6,4


In [8]:
company_count

[134, 55, 22, 23, 13, 11, 8, 6, 4, 3]

In [9]:
bar = Bar('城市分布', subtitle='城市-职位数/公司数')
bar.add('职位数量', x_axis=city_top10, xaxis_interval=0, bar_category_gap=0,
        y_axis=job_count, is_label_show=1)
bar.add('公司数量', is_label_show=1,
        x_axis=city_top10, xaxis_interval=0, xaxis_rotate=-45,
        y_axis=company_count, yaxis_name='职位数量', yaxis_force_interval=50,
        yaxis_name_gap=40, datazoom_orient = 'horizontal',
        datazoom_type = 'both', is_stack=0, bar_category_gap=20)

line = Line()
line.add('',x_axis=city_top10, y_axis=job_per_com, line_width=2,
        xaxis_interval=0, is_label_show=1,
        yaxis_max=2.5, yaxis_force_interval=0.5)

overlap = Overlap()
overlap.add(bar)
overlap.add(line, yaxis_index=1, is_add_yaxis=True)
overlap

In [10]:
locate = []
jcount = []
ccount = []
elsesum_j = 0
elsesum_c = 0
for i in range(0,len(city_top10)):
    if job_count[i] >= 15:
        locate.append(city_top10[i])
        jcount.append(job_count[i])
        ccount.append(company_count[i])
    else:
        elsesum_j += job_count[i]
        elsesum_c += company_count[i]
locate.append('其他')
jcount.append(elsesum_j)
ccount.append(elsesum_c)

pie = Pie('城市占比',subtitle = '城市-职位数',title_pos = 'center',title_top = '44%')
pie.add('',locate,jcount,radius=[40,75],is_label_show = 1)
pie

In [11]:
# 前五的城市,每个城市的公司(top10)
citys= ['北京', '上海', '杭州', '深圳', '广州']
for c in citys:
    df = data[data['city']==c].groupby('companyShortName').agg({'count':pd.Series.count})\
        .sort_values(['count'], ascending=[0])
    print([c, list(df.index)[:5]])

['北京', ['美团点评', '京东商城', '小米', '搜狗', '好未来']]
['上海', ['上海宏鹿信息技术服务有限公司', '网贷之家', '极光', '上海晋景数据', '华院数据']]
['杭州', ['蚂蚁金服集团', '微拍堂', '浙江执御信息技术有限公司', '浙报集团', '魔蝎科技']]
['深圳', ['无秘', '随手科技', '试金石信用', '乐信', '极光']]
['广州', ['广州银行信用卡中心', '蓝盾信息安全技术股份有限公司', 'Bigo', 'Edison Inc', 'vivo']]


In [12]:
companySize = data.groupby('companySize').agg({'positionName': pd.Series.count,'companyShortName': pd.Series.nunique})
companySize = companySize.sort_values(['positionName', 'companyShortName'], ascending=[0, 0])
companySize_top10 = list(companySize.index)[:10]
job_count = list(map(int, companySize['positionName']))[:10]   # 职位计数
company_count = list(map(int, companySize['companyShortName']))[:10]   # 公司计数
job_per_com = []    
for i in range(0, len(companySize_top10)):
    job_per_com.append(round(job_count[i]/company_count[i], 2))
companySize

,positionName,companyShortName
companySize,,
2000人以上,161,63
150-500人,113,81
500-2000人,93,59
50-150人,61,48
15-50人,19,18
少于15人,3,2


In [13]:
company_count

[63, 81, 59, 48, 18, 2]

In [15]:
bar = Bar('规模分布','公司规模-职位数/公司数')
bar.add('职位数量', x_axis=companySize_top10, xaxis_interval=0, bar_category_gap=20,
        y_axis=job_count, is_label_show=1)
bar.add('公司数量', is_label_show=1,
        x_axis=companySize_top10, xaxis_rotate=-45,
        y_axis=company_count, yaxis_name='职位数量', yaxis_force_intervel=30,yaxis_name_gap=40,
        datazoom_orient = 'horizontal',datazoom_type = 'both',
        bar_category_gap=20)

line = Line()
line.add('',x_axis=companySize_top10,y_axis=job_per_com,line_width=2,is_label_show=True)
overlap = Overlap()
overlap.add(bar)
overlap.add(line,yaxis_index=1,is_add_yaxis=True)
overlap

In [16]:
size = []
jcount = []
ccount = []
for i in range(0,len(companySize_top10)):
    if companySize_top10[i] != 'null':
        size.append(companySize_top10[i])
        jcount.append(int(job_count[i]))
        ccount.append(int(company_count[i]))

In [17]:
pie = Pie('规模占比',subtitle = '公司规模-职位数',title_pos = 'center',title_top = '44%')
pie.add('',size,jcount,radius=[40,75],is_label_show = 1)
pie

In [18]:
for size in companySize_top10:
    df = data[data['companySize']==size].groupby('companyShortName').agg({'count':pd.Series.count})\
    .sort_values(['count'],ascending=[0])
    print([size,list(df.index)[:5]])

['2000人以上', ['美团点评', '蚂蚁金服集团', '京东商城', '小米', '金山办公软件']]
['150-500人', ['无秘', '集奥聚合（GEO）', '极光', '数美', '网贷之家']]
['500-2000人', ['今日头条', '奇虎360金融', '美图公司', '新华网', 'Gridsum 国双']]
['50-150人', ['上海宏鹿信息技术服务有限公司', '最右', '商询科技', '试金石信用', '慕课网']]
['15-50人', ['小库科技', 'ARKie', '深睿医疗', '阡寻', '轻趣']]
['少于15人', ['上海晋景数据', '北京跬步众成信息技术有限公司']]


In [19]:

from pyecharts import WordCloud
def count_word(word_list):
    '''
    词频统计
    '''
    cnt = dict()
    for word in word_list:
        if word != 'null':
            if word in cnt.keys():
                cnt[word] += 1
            else:
                cnt[word] = 1
    return cnt


def wordcloud(word_list):
    dic = count_word(word_list)
    word = list(dic.keys())
    freq = list(dic.values())
    wordc = WordCloud(width=900, height=450)
    wordc.add("福利", word, freq, word_size_range=[15,100], shape='circle')
    #输出top10及词频
    sort_dic = sorted(dic.items(), key = lambda asd:asd[1],reverse = True)
    for i in range(0,10):
        print(i+1,sort_dic[i])
    return wordc

In [20]:
#tag_词云
df = data[data['companyLabelList'] != '']['companyLabelList']
# print(list(df))
tag_list = list(df)
tag_list_all = []
for i in range(0,len(tag_list)):
    tag_split = tag_list[i].split(',')
    for each in tag_split:
        if each != '':
            tag_list_all.append(each)
wordcloud(tag_list_all)

1 ('带薪年假', 170)
2 ('绩效奖金', 141)
3 ('股票期权', 121)
4 ('技能培训', 108)
5 ('节日礼物', 107)
6 ('年底双薪', 98)
7 ('岗位晋升', 95)
8 ('五险一金', 92)
9 ('扁平管理', 68)
10 ('年度旅游', 53)


In [21]:
#判断str是否含有中文 True or False
def check_contain_chinese(check_str):
    for ch in check_str:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False
#字符筛选器：包含返回True，否则返回False
def if_contains(s):
    value = False
    filter = ['数据','岗位','学历','互联网','计算机','全日制','and','熟练','责任心']
    for each in filter:
        if each in s:
            value = True
    return value

In [22]:
#合并description字段->一个str，分词，并输出词云以及词频top10
import jieba 
def des_cloud(des_list):
    description_all = ''
    for each in des_list:
            if each != '':
                description_all += each+','
    seg_list = jieba.cut(description_all,cut_all=True)
    seg = []
    for i in seg_list:
        if check_contain_chinese(i):
            if len(i)>2 and not if_contains(i):
                seg.append(i)
        else:
            if ((len(i) > 1 and i.isdigit() != True) or (i == 'R')) and (not if_contains(i)):
                seg.append(str.upper(i)) 
    wordc = wordcloud(seg)
    return wordc

In [23]:
#所需技术——关键词
df = data[(data['job_description'] != '')]['job_description']
des_list = list(df)
des_cloud(des_list)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.214 seconds.
Prefix dict has been built succesfully.


1 ('PYTHON', 370)
2 ('HADOOP', 259)
3 ('SPARK', 233)
4 ('JAVA', 225)
5 ('R', 155)
6 ('统计学', 147)
7 ('HIVE', 143)
8 ('分布式', 139)
9 ('SQL', 132)
10 ('LINUX', 116)
